<a href="https://colab.research.google.com/github/abrham17/ML_Pytorch_repo/blob/main/amharic_sentiment_analysis_fine_tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets
!pip install torch
import torch , transformers , datasets ,  pandas , sklearn , numpy

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load pre-trained tokenizer and model fine-tuned for sentiment analysis
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-uncased")



In [4]:
import pandas as pd

df = pd.read_csv('/content/filteded.csv')


In [5]:
from datasets import Dataset
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2)
train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)
print(train_dataset.column_names)


['clean_tweet', 'label', '__index_level_0__']


In [25]:
def tokenize_function(examples):
    return tokenizer(examples["clean_tweet"], padding="max_length", truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

input_ids = train_dataset[0]["input_ids"]

decoded_text = tokenizer.decode(input_ids, skip_special_tokens=True)

print("Decoded text:", decoded_text)


Map:   0%|          | 0/1031 [00:00<?, ? examples/s]

Map:   0%|          | 0/258 [00:00<?, ? examples/s]

Decoded text: 


In [10]:
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
label_map = {"positive" : 2 , "negative" : 0 , "neutral" : 1}
train_dataset = train_dataset.map(lambda x : {"label" : label_map[x["label"]]})
test_dataset = test_dataset.map(lambda x : {"label" : label_map[x["label"]]})

Map:   0%|          | 0/1031 [00:00<?, ? examples/s]

Map:   0%|          | 0/258 [00:00<?, ? examples/s]

In [26]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-uncased", num_labels=3)
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,1.229200
20,1.153400
30,0.970300
40,0.669900
50,0.336000
60,0.143400
70,0.062100
80,0.030600
90,0.017800
100,0.011700


TrainOutput(global_step=387, training_loss=0.12105838653272936, metrics={'train_runtime': 319.9139, 'train_samples_per_second': 9.668, 'train_steps_per_second': 1.21, 'total_flos': 813809801032704.0, 'train_loss': 0.12105838653272936, 'epoch': 3.0})